In [2]:
import os
import re
import json
import pandas as pd
from tqdm import tqdm
import torch
import gc
import sys
import traceback

# --- Unsloth/Transformers Imports ---
from unsloth import FastLanguageModel
# from transformers import TextStreamer # Not needed for this version
from unsloth.chat_templates import get_chat_template

# --- Scikit-learn Metrics Imports ---
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score
)

# --- Configuration ---
# <<< *** SET THIS TO YOUR SAVED ADAPTER DIRECTORY *** >>>
FINETUNED_MODEL_DIR = "gemma3_train_with_GPT4.1"
# <<< Path to your test CSV (must have 'text' and 'labels' columns) >>>
INPUT_CSV = "../data/test.csv"
# <<< Output directory and filename prefix for results >>>
OUTPUT_DIR = "finetuned_model_results" # Changed directory name
RESULTS_FILENAME = "output_train_with_GPT" # Changed filename prefix

# --- Helper Functions ---
def fix_tweet_text(tweet: str) -> str:
    """Clean up tweets by removing extra whitespace."""
    if not isinstance(tweet, str):
        tweet = str(tweet)
    return re.sub(r'\s+', ' ', tweet).strip()

# --- Main Execution ---
def main():
    # 1. Load the Fine-Tuned Model and Tokenizer
    print(f"Loading fine-tuned model from: {FINETUNED_MODEL_DIR}")
    try:
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name = FINETUNED_MODEL_DIR,
            max_seq_length = 2048,
            load_in_4bit = True,
            load_in_8bit = False,
            dtype = None,
            device_map = "auto",
        )
        print("Model loaded successfully.")

        print("Applying chat template to tokenizer...")
        tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")
        print("Tokenizer ready.")

    except Exception as e:
        print(f"\n--- ERROR DURING MODEL LOADING ---"); print(f"Error message: {e}")
        print(f"\nEnsure the directory '{FINETUNED_MODEL_DIR}' exists and contains valid adapter files.")
        print("Check VRAM availability (nvidia-smi)."); traceback.print_exc(); sys.exit(1)

    # 2. Load Test Data
    try:
        test_df = pd.read_csv(INPUT_CSV)
        if 'text' not in test_df.columns or 'labels' not in test_df.columns:
            raise ValueError("CSV must contain 'text' and 'labels' columns.")
        print(f"Loaded {len(test_df)} tweets from {INPUT_CSV}")
    except FileNotFoundError: print(f"Error: Input file not found at {INPUT_CSV}"); return
    except Exception as e: print(f"Error loading CSV: {e}"); return

    # 3. Prepare Output Directory
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # 4. Process Tweets and Collect Results using Fine-tuned Model
    results_data = []
    all_actual_labels = []
    all_predicted_labels = []

    inference_instruction = """    I am a researcher interested in studying disaster response events. My goal is to analyze historical tweets to better understand responses to various types of disasters.

Each tweet I receive includes a disaster-related keyword or hashtag, but not all tweets describe an actual disaster event. I need a model to classify tweets into two categories based on whether they are truly disaster-related:
1 - Disaster-Related (tweets that report actual emergencies, accidents, crises, or factual news about such events requiring attention).
0 - Not Disaster-Related (tweets that use disaster-related terms metaphorically, humorously, or in other non-emergency contexts).

Consider these guidelines for classification:
- Tweets reporting actual emergencies, accidents, or crises (such as natural disasters, transportation accidents, industrial incidents, or health emergencies) should be classified as 1.
- Factual news reports about any actual disaster or accident should be classified as 1.
- Tweets using disaster-related terminology metaphorically, humorously, or in a non-urgent manner should be classified as 0.
- Jokes, memes, or casual mentions that do not indicate real-life emergencies should be classified as 0.

Please return the classification (0 or 1) followed by a brief reasoning for your decision. These two parts—classification and reasoning—should be returned in one line, separated by a single semicolon (;).

For example:
1; This tweet mentions rescue efforts and evacuation details following a reported flood.

Please return your answer in this exact format:
classification; reasoning
where classification is either 0 or 1, and reasoning is a short explanation.

Important: When you write the reasoning, do not use any semicolons (;) within the reasoning text itself, as your answer will be processed by a script expecting only one semicolon separator in the entire line.


Tweet: """

    print(f"\n=== Classifying tweets using fine-tuned model from {FINETUNED_MODEL_DIR} ===")
    model.eval()

    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Classifying", unit="tweet"):
        tweet = row['text']
        try:
            actual_label = int(row['labels'])
            if actual_label not in [0, 1]: raise ValueError("Actual label must be 0 or 1")
        except (ValueError, TypeError) as e:
            print(f"\nSkipping row due to invalid actual label: {row['labels']} ({e}). Tweet: {str(tweet)[:50]}...")
            results_data.append({"tweet": fix_tweet_text(tweet), "predicted_label": "error", "reasoning": "", "actual_label": row['labels'], "raw_response": "", "error": f"Invalid actual label: {e}"})
            continue

        tweet_clean = fix_tweet_text(tweet)
        pred_label_parsed = "error"
        reasoning_parsed = ""
        raw_resp_decoded = ""
        error_msg = None

        try:
            user_content = f"{inference_instruction}{tweet_clean}"

            # --- **** MESSAGE STRUCTURE CHANGE HERE **** ---
            messages = [
                {
                    "role": "user",
                    "content": [ # Content is now a list...
                        {"type": "text", "text": user_content} # ...containing a dictionary for the text block
                    ]
                }
            ]
            # --- **** END MESSAGE STRUCTURE CHANGE **** ---

            inputs = tokenizer.apply_chat_template(
                messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt",
            ).to("cuda" if torch.cuda.is_available() else "cpu")

            with torch.no_grad():
                 outputs = model.generate(
                    input_ids = inputs, max_new_tokens = 75, do_sample = False,
                    pad_token_id = tokenizer.eos_token_id
                )

            input_length = inputs.shape[1]
            generated_ids = outputs[0][input_length:]
            raw_resp_decoded = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

            parts = raw_resp_decoded.split(";")
            parts = [p.strip() for p in parts]

            if len(parts) != 2:
                if parts and parts[0].lower() == 'error': # Check if parts is not empty before indexing
                      raise ValueError(f"Ollama API or internal error: {parts[1] if len(parts)>1 else raw_resp_decoded}")
                else:
                      raise ValueError(f"Expected 2 parts separated by ';' but got {len(parts)}. Response: '{raw_resp_decoded}'")

            pred_label_parsed = int(parts[0])
            if pred_label_parsed not in [0, 1]:
                raise ValueError(f"Predicted label must be 0 or 1, but got {pred_label_parsed}")

            reasoning_parsed = parts[1]

            all_actual_labels.append(actual_label)
            all_predicted_labels.append(pred_label_parsed)

        except Exception as e:
            error_msg = f"Inference or Parsing error: {e}"
            # --- Adding detailed debug from previous step (Optional but helpful) ---
            print(f"\n--- Debug: Error occurred ---")
            print(f"Tweet: {tweet_clean[:100]}...")
            print(f"Raw Response Decoded: '{raw_resp_decoded}'")
            print(f"Exception Type: {type(e)}")
            print(f"Exception Args: {e.args}")
            print("Traceback:")
            traceback.print_exc()
            print(f"--- End Debug ---")
            # --- End detailed debugging ---

        results_data.append({
            "tweet": tweet_clean, "predicted_label": pred_label_parsed, "reasoning": reasoning_parsed,
            "actual_label": actual_label, "raw_response": raw_resp_decoded, "error": error_msg
        })

    # 5. Save detailed results
    results_df = pd.DataFrame(results_data)
    csv_path = os.path.join(OUTPUT_DIR, f"{RESULTS_FILENAME}_details.csv")
    json_path = os.path.join(OUTPUT_DIR, f"{RESULTS_FILENAME}_details.json")
    try:
        results_df.to_csv(csv_path, index=False)
        results_df.to_json(json_path, orient='records', indent=2)
        print(f"\nDetailed results saved to {csv_path} and {json_path}")
    except Exception as e: print(f"Error saving detailed results: {e}")

    # 6. Calculate and Print Metrics
    print("\n=== Classification Metrics ===")
    if not all_actual_labels:
        print("No valid predictions were successfully processed. Cannot calculate metrics.")
        del model, tokenizer; gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None; return

    try:
        all_actual_labels_int = [int(l) for l in all_actual_labels]
        all_predicted_labels_int = [int(l) for l in all_predicted_labels]
    except ValueError:
        print("Error: Could not convert all labels to integers for metric calculation.")
        del model, tokenizer; gc.collect(); torch.cuda.empty_cache() if torch.cuda.is_available() else None; return

    # -- Confusion Matrix --
    print("\nConfusion Matrix:")
    try:
        cm = confusion_matrix(all_actual_labels_int, all_predicted_labels_int, labels=[0, 1]) # Ensure labels are 0, 1
        print(pd.DataFrame(cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1']))
    except Exception as e: print(f"Error calculating/displaying Confusion Matrix: {e}")

    # -- Classification Report --
    print("\nClassification Report:")
    try:
        report = classification_report(
            all_actual_labels_int, all_predicted_labels_int,
            labels = [0, 1], # Ensure labels are 0, 1
            target_names=['Not Disaster-Related (0)', 'Disaster-Related (1)'],
            digits=4, zero_division=0
        )
        print(report)
    except Exception as e: print(f"Error calculating Classification Report: {e}")

    print(f"\nEvaluation complete. Check '{OUTPUT_DIR}/' for detailed results.")

    # Optional: Clean up model from memory at the very end
    print("\nCleaning up GPU memory...")
    del model, tokenizer; gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    print("Cleanup complete.")

if __name__ == "__main__":
    main()

Loading fine-tuned model from: gemma3_train_with_GPT4.1
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully.
Applying chat template to tokenizer...
Tokenizer ready.
Loaded 1523 tweets from ../data/test.csv

=== Classifying tweets using fine-tuned model from gemma3_train_with_GPT4.1 ===


Classifying:   0%|▏                                                                 | 3/1523 [00:03<33:02,  1.30s/tweet]Traceback (most recent call last):
  File "/tmp/ipykernel_2751/1298950190.py", line 160, in main
    raise ValueError(f"Expected 2 parts separated by ';' but got {len(parts)}. Response: '{raw_resp_decoded}'")
ValueError: Expected 2 parts separated by ';' but got 3. Response: '0; This tweet expresses gratitude for a successful event and doesn't report any actual disaster or emergency; it uses disaster-related terms metaphorically to describe a positive experience.'
Classifying:   0%|▏                                                                 | 4/1523 [00:05<38:23,  1.52s/tweet]


--- Debug: Error occurred ---
Tweet: Aftershock back to school kick off was great. I want to thank everyone for making it possible. What ...
Raw Response Decoded: '0; This tweet expresses gratitude for a successful event and doesn't report any actual disaster or emergency; it uses disaster-related terms metaphorically to describe a positive experience.'
Exception Type: <class 'ValueError'>
Exception Args: ("Expected 2 parts separated by ';' but got 3. Response: '0; This tweet expresses gratitude for a successful event and doesn't report any actual disaster or emergency; it uses disaster-related terms metaphorically to describe a positive experience.'",)
Traceback:
--- End Debug ---


Classifying:   3%|██                                                               | 47/1523 [01:10<33:29,  1.36s/tweet]Traceback (most recent call last):
  File "/tmp/ipykernel_2751/1298950190.py", line 160, in main
    raise ValueError(f"Expected 2 parts separated by ';' but got {len(parts)}. Response: '{raw_resp_decoded}'")
ValueError: Expected 2 parts separated by ';' but got 3. Response: '0; This tweet does not report a real-life disaster or emergency; it's about removing links and a browser hijacker.'
Classifying:   3%|██                                                               | 48/1523 [01:12<33:08,  1.35s/tweet]


--- Debug: Error occurred ---
Tweet: Remove the http://t.co/Xxj2B4JxRt and Linkury Browser Hijacker http://t.co/9gtYlgXrOE http://t.co/yG...
Raw Response Decoded: '0; This tweet does not report a real-life disaster or emergency; it's about removing links and a browser hijacker.'
Exception Type: <class 'ValueError'>
Exception Args: ("Expected 2 parts separated by ';' but got 3. Response: '0; This tweet does not report a real-life disaster or emergency; it's about removing links and a browser hijacker.'",)
Traceback:
--- End Debug ---


...


--- Debug: Error occurred ---
Tweet: The mixtape is coming i promise. We goin in right now http://t.co/uUNGRqoUgn...
Raw Response Decoded: '0; This tweet does not relate to any actual disaster or emergency; it’s a metaphorical statement about releasing a mixtape.'
Exception Type: <class 'ValueError'>
Exception Args: ("Expected 2 parts separated by ';' but got 3. Response: '0; This tweet does not relate to any actual disaster or emergency; it’s a metaphorical statement about releasing a mixtape.'",)
Traceback:
--- End Debug ---


Classifying:  99%|██████████████████████████████████████████████████████████████▌| 1513/1523 [36:24<00:14,  1.47s/tweet]Traceback (most recent call last):
  File "/tmp/ipykernel_2751/1298950190.py", line 160, in main
    raise ValueError(f"Expected 2 parts separated by ';' but got {len(parts)}. Response: '{raw_resp_decoded}'")
ValueError: Expected 2 parts separated by ';' but got 3. Response: '0; This tweet does not contain any information related to a disaster or emergency; it is simply a question about a missing object.'
Classifying:  99%|██████████████████████████████████████████████████████████████▋| 1514/1523 [36:26<00:12,  1.43s/tweet]


--- Debug: Error occurred ---
Tweet: 'wHeRE's mY aRsOnISt aT???'...
Raw Response Decoded: '0; This tweet does not contain any information related to a disaster or emergency; it is simply a question about a missing object.'
Exception Type: <class 'ValueError'>
Exception Args: ("Expected 2 parts separated by ';' but got 3. Response: '0; This tweet does not contain any information related to a disaster or emergency; it is simply a question about a missing object.'",)
Traceback:
--- End Debug ---


Classifying: 100%|███████████████████████████████████████████████████████████████| 1523/1523 [36:39<00:00,  1.44s/tweet]



Detailed results saved to finetuned_model_results/output_train_with_GPT_details.csv and finetuned_model_results/output_train_with_GPT_details.json

=== Classification Metrics ===

Confusion Matrix:
          Predicted 0  Predicted 1
Actual 0          608          212
Actual 1          129          506

Classification Report:
                          precision    recall  f1-score   support

Not Disaster-Related (0)     0.8250    0.7415    0.7810       820
    Disaster-Related (1)     0.7047    0.7969    0.7480       635

                accuracy                         0.7656      1455
               macro avg     0.7649    0.7692    0.7645      1455
            weighted avg     0.7725    0.7656    0.7666      1455


Evaluation complete. Check 'finetuned_model_results/' for detailed results.

Cleaning up GPU memory...
Cleanup complete.
